In [ ]:
!pip install transformers torch pandas numpy scipy tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import pandas as pd
from scipy.special import softmax
from tqdm import tqdm

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model and tokenizer
MODEL_NAME = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)  # Move model to GPU
model.eval()

# Load dataset
data = pd.read_csv('/content/drive/MyDrive/Module_2/data_set/processed_data/cleaned_text.csv')

# Convert column to string to avoid errors
data['text'] = data['text'].astype(str)

emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

def get_dominant_emotion(texts):
    """Predict the dominant emotion for a batch of texts."""
    if not isinstance(texts, list):
        texts = [texts]  # Ensure input is always a list

    inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors='pt')

    # Move inputs to GPU
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    scores = outputs.logits.cpu().numpy()  # Move logits to CPU
    scores = softmax(scores, axis=1)  # Apply softmax to get probabilities

    # Debugging prints
    print(f"Scores shape: {scores.shape}, Expected labels: {len(emotion_labels)}")

    # Ensure the number of output labels matches expectation
    if scores.shape[1] != len(emotion_labels):
        raise ValueError(f"Model output {scores.shape[1]} labels, expected {len(emotion_labels)}")

    dominant_indices = np.argmax(scores, axis=1)  # Get index of max probability

    # Ensure valid indices
    valid_indices = np.clip(dominant_indices, 0, len(emotion_labels) - 1)

    return [emotion_labels[idx] for idx in valid_indices]

# Process emotions in batches
BATCH_SIZE = 32
dominant_emotions = []

for i in tqdm(range(0, len(data), BATCH_SIZE), desc="Processing Emotions"):
    batch_texts = data['text'][i:i+BATCH_SIZE].tolist()
    batch_texts = [str(text) for text in batch_texts]
    batch_dominant = get_dominant_emotion(batch_texts)
    dominant_emotions.extend(batch_dominant)

data['Detected Emotion'] = dominant_emotions

data[['text','label', 'Detected Emotion']].to_csv('/content/drive/MyDrive/Module_2/data_set/processed_data/emotions.csv', index=False)

print("✅ Emotion Detection Completed & Saved with ONLY Detected Emotion!")


Using device: cuda


Processing Emotions:   0%|          | 2/2383 [00:00<08:14,  4.82it/s]

Scores shape: (32, 7), Expected labels: 7
Scores shape: (32, 7), Expected labels: 7


Processing Emotions:   0%|          | 4/2383 [00:00<06:57,  5.70it/s]

Scores shape: (32, 7), Expected labels: 7
Scores shape: (32, 7), Expected labels: 7


...

Processing Emotions: 100%|█████████▉| 2381/2383 [04:52<00:00,  8.29it/s]

Scores shape: (32, 7), Expected labels: 7
Scores shape: (32, 7), Expected labels: 7


Processing Emotions: 100%|██████████| 2383/2383 [04:52<00:00,  8.13it/s]

Scores shape: (32, 7), Expected labels: 7
Scores shape: (8, 7), Expected labels: 7


✅ Emotion Detection Completed & Saved with ONLY Detected Emotion!


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Module_2/data_set/processed_data/emotions.csv')
df

,text,label,Detected Emotion
0,im a single person with an overflowing hoarder...,hoarding disorder,sadness
1,my grandmother has been a hoarder her entire l...,hoarding disorder,sadness
2,hoarding itself is not such a bad thing its th...,hoarding disorder,anger
3,i know eight hoarders i met them in my travels...,hoarding disorder,sadness
4,gad generalised anxiety disorder is the most c...,hoarding disorder,fear
...,...,...,...
76227,the difference between a narcissist and a psyc...,personality disorder,fear
76228,hey everyone looking for thoughtsadviceassista...,anxiety,fear
76229,what if i were god and i would ask you to dema...,eating disorder,sadness
76230,i want to kill her and him i want to watch her...,bpd,sadness
